In [1]:
import time

import numpy as np
import datetime

"""시뮬레이션 결과파일에서 데이터 가져오기 (중차량 비율 20~35%)"""
import sqlite3
import pandas as pd
import win32com.client  # Windows COM 인터페이스 사용
import os
import xml.etree.ElementTree as ET
####################### 입력 값 ############################
# 폴더 경로 설정
folder_path = r"C:\VISSIM_Workspace\network_new\network_version2\network_version4.1"  # 폴더 경로 입력 받아야함 추후에 함수에 파라미터로 처리 예정
# 웜-업 타임
from_time = 1800
# 진입부, 출입부 링크 번호
link_no = "1" # 파라미터 처리
link_no2 = "3"
# 시뮬레이션 횟수
siml_count = 10

# 시나리오 갯수
senario_cnt = 36 # 36 (수정해야함)

# 중차량 반복 값
relflow_start = 20  # 시작(%) # 20 (수정해야함)
relflow_final = 30 # 마지막(%)
relflow_interval = 5 # 간격(%)

# 종단경사 반복 값
gradient_start = 3  # 시작(%) (수정해야함)
gradient_final = 9 # 마지막(%)
gradient_interval = 2 # 간격(%)

# 교통량
volume_list = np.array([
    [1388, 1378, 1368, 1356, 1346, 1336],
    [2976, 2952, 2930, 2906, 2884, 2862],
    [3770, 3740, 3710, 3682, 3654, 3626]
])

# 차량 비율
vehicle1 = np.array([86.24, 81.70, 77.16, 72.62, 68.09, 63.55]) # 소형차
vehicle2 = np.array([8.76, 8.30, 7.84, 7.38, 6.91, 6.45])   # 버스
vehicle3 = np.array([3.56, 7.13, 10.69, 14.26, 17.82, 21.38])   # 소형화물
vehicle4 = np.array([1.27, 2.53, 3.80, 5.07, 6.34, 7.60])   # 중형화물
vehicle5 = np.array([0.17, 0.34, 0.51, 0.68, 0.84, 1.01])   # 대형화물
###########################################################
# 폴더 내 파일 목록 가져오기
file_list = os.listdir(folder_path)
inpx_files = [file for file in file_list if file.endswith(".inpx")]

# VISSIM 실행 및 연결
Vissim = win32com.client.Dispatch("Vissim.Vissim")  # Vissim 인스턴스 생성
simulation_run_count = 0  # VISSIM 실행 횟수 카운트
senario_start = 0 #   수정

for i in range(len(inpx_files)):
    file_name = inpx_files[i]
    file_path = folder_path + "\\" +file_name
    print("file_path : ", file_path)
    for v in range(len(volume_list)):
        for r in range(relflow_start,relflow_final+1,relflow_interval):
            a = int(r/relflow_interval-1)
            veh = np.array([vehicle1[a], vehicle2[a], vehicle3[a], vehicle4[a], vehicle5[a]]) * 0.01
            volume = volume_list[v][a]

            for g in range(gradient_start,gradient_final+1, gradient_interval):
                # 종단경사 설정
                new_gradient = g * -0.01
                new_gradient2 = g * 0.01


                # XML 파일 파싱
                tree = ET.parse(file_path)
                root = tree.getroot()

                # <link> 태그 찾기
                for link in root.findall(".//link"):
                    if link.get("no") == str(link_no):  # 특정 no 값을 가진 <link> 찾기
                        link.set("gradient", str(new_gradient))  # gradient 속성 변경
                    if link.get("no") == str(link_no2):  # 특정 no 값을 가진 <link> 찾기
                        link.set("gradient", str(new_gradient2))  # gradient 속성 변경

                    # <vehicleCompositions> 태그 내 <vehicleComposition name="Default"> 찾기
                for vehicle_comp in root.findall(".//vehicleComposition"):
                    if vehicle_comp.get("name") == "com1":

                        # <vehCompRelFlows> 태그 찾기
                        vehCompRelFlows = vehicle_comp.find("vehCompRelFlows")
                        if vehCompRelFlows is not None:
                            # <vehicleCompositionRelativeFlow> 태그 찾기
                            for relflow in vehCompRelFlows.findall("vehicleCompositionRelativeFlow"):
                                vehType = relflow.get("vehType")
                                if(vehType == "100"): # 승용차
                                    relflow.set("relFlow", str(round(veh[0],4)))
                                if(vehType == "300"): # 버스
                                    relflow.set("relFlow", str(round(veh[1],4)))
                                if(vehType == "630"): # 소형화물
                                    relflow.set("relFlow", str(round(veh[2],4)))
                                if(vehType == "640"): # 중형화물
                                    relflow.set("relFlow", str(round(veh[3],4)))
                                if(vehType == "650"): # 대형화물
                                    relflow.set("relFlow", str(round(veh[4],4)))
                for vehicle_input in root.findall(".//vehicleInput"):
                    if vehicle_input.get("no") == "1":
                        # <timeIntVehVols> 태그 찾기
                        vehicleInput = vehicle_input.find("timeIntVehVols")
                        if vehicleInput is not None:
                            for vol in vehicleInput.findall("timeIntervalVehVolume"):
                                vol.set("volume", str(round(volume)))
                # 변경된 XML을 원본 파일에 덮어쓰기
                tree.write(file_path, encoding="utf-8", xml_declaration=True)

                print("new_gradient : ", new_gradient, "new_gradient2 : ", new_gradient2)
                print("100rel: ", round(veh[0],4), "300rel : ", round(veh[1],4), "630rel : ", round(veh[2],4), "640rel : ", round(veh[3],4), "650rel : ", round(veh[4],4), "volume : ", volume)

                # VISSIM 네트워크 로드
                Vissim.LoadNet(file_path)
                Vissim.Graphics.CurrentNetworkWindow.SetAttValue('QuickMode', 1) # 퀵 모드 활성화

                print("VISSIM 시뮬레이션 실행 중...")
                Vissim.Simulation.RunContinuous()

                Vissim.Simulation.Stop()
                print("VISSIM 시뮬레이션 종료")

                #Vissim.exit()

                #del Vissim

                interval_value = 0

                # dataColl 태그에서 interval 속성 값 찾기
                for elem in root.iter("dataColl"):
                    interval_value = int(elem.get("interval"))  # "interval" 속성 값 가져오기
                    """
                    if interval_value:
                        print(f"추출된 interval 값: {interval_value}") # 60
                    else:
                        print("추출된 interval 값이 없습니다.")
                    """
                # 데이터베이스 파일 경로
                # "C:/VISSIM_Workspace/test/test.results/3.db"
                # 확장자 제거
                results_path = folder_path + "\\" +os.path.splitext(file_name)[0] + ".results"
                results_files = sorted([f for f in os.listdir(results_path) if f.endswith(".db")], key=lambda x: int(x.split(".")[0]))
                print("results_files : ", results_files)

                senario_end = senario_start + siml_count


                # 시뮬레이션 갯수만큼 묶어서 처리(1/2)
                for start in range(senario_start, senario_end, siml_count):
                    batch_files = results_files[start:start + siml_count]  # 시뮬레이션 횟수 만큼 분할
                    print("batch files : ", batch_files)
                    df_list = []
                    for b in range(len(batch_files)):
                        db_path = os.path.join(results_path, batch_files[b])
                        print("db_path : ", db_path)
                        # 데이터베이스 연결
                        with sqlite3.connect(db_path) as conn:
                            df = pd.read_sql_query("SELECT * FROM DATACOLLECTIONMEASUREMENT_EvaluationTimeIntervalClass;", conn)
                        df_list.append(df)
                    # 인덱스별 평균값 구하기
                    df = pd.concat(df_list).groupby(level=0).mean()

                    #.groupby(level=0).mean() 실행 시 문자형 데이터는 잘리기 때문에
                    df[["ARG_TIMEINTERVAL", "OBJECT_ID"]] = df_list[0][["ARG_TIMEINTERVAL", "OBJECT_ID"]]

                    # TIMEINT 변환
                    df["ARG_TIMEINTERVAL"] = from_time + df["ARG_TIMEINTERVAL"].astype(int) * interval_value   # 변환 시작점 보정

                    # OCCUPRATE 보정
                    df["OCCUPRATE"] = df["OCCUPRATE"].astype(float)*100
                    df["OCCUPRATE"] = df["OCCUPRATE"].apply(lambda x: f"{x:.2f}")  # 항상 소수점 둘째 자리까지 유지
                    df = df.round(2)
                    df["OCCUPRATE"] = df["OCCUPRATE"].astype(str) + "%"

                    # SIMRUN 설정
                    df["SIMRUN"] =siml_count
                    df["OBJECT_ID"] = df["OBJECT_ID"].astype(int)
                    # 컬럼명 변경
                    df.rename(columns={
                        "ARG_TIMEINTERVAL": "TIMEINT",
                        "OBJECT_ID": "DATACOLLECTIONMEASUREMENT",
                        "ACCELERATION" : "ACCELERATION(ALL)",
                        "DIST" : "DIST(ALL)",
                        "LENGTH" : "LENGTH(ALL)",
                        "VEHS" : "VEHS(ALL)",
                        "PERS" : "PERS(ALL)",
                        "QUEUEDELAY" : "QUEUEDELAY(ALL)",
                        "SPEEDAVGARITH" : "SPEEDAVGARITH(ALL)",
                        "SPEEDAVGHARM" : "SPEEDAVGHARM(ALL)",
                        "OCCUPRATE" : "OCCUPRATE(ALL)"
                    }, inplace=True)

                    # 밀도 컬럼 추가
                    df["K"] = (df["VEHS(ALL)"] * 60 / df["SPEEDAVGARITH(ALL)"]).round(2)

                    # 순서 재정의
                    df = df[["SIMRUN", "TIMEINT", "DATACOLLECTIONMEASUREMENT", "ACCELERATION(ALL)", "DIST(ALL)",
                             "LENGTH(ALL)", "VEHS(ALL)", "PERS(ALL)", "QUEUEDELAY(ALL)", "SPEEDAVGARITH(ALL)", "SPEEDAVGHARM(ALL)", "OCCUPRATE(ALL)", "K"]]

                    excel_dir  = os.path.join(folder_path, os.path.splitext(file_name)[0] + "_output")
                    if not os.path.exists(excel_dir):
                        os.makedirs(excel_dir)
                    last_file = xlsx_files = sorted([f for f in os.listdir(excel_dir) if f.endswith(".xlsx")], key=lambda x: int(x.split("_")[0]) if x.split("_")[0].isdigit() else 0)

                    if not xlsx_files: # 폴더에 파일이 존재하지 않을 때의 시작 번호
                        last_file = 13
                    else:
                        last_file = int(xlsx_files[-1].split("_")[0]) + 1  # 가장 마지막 파일 번호 + 1
                    excel_path = os.path.join(excel_dir, f"{last_file}_output.xlsx")
                    df.to_excel(excel_path, index=False)
                    print(f"생성파일: {excel_path}")

                    # VISSIM 실행 횟수 증가
                    simulation_run_count += 1
                    # senario_start 번호 증가
                    senario_start += siml_count

                    # **10번 실행할 때마다 VISSIM 재시작**
                    if simulation_run_count % 4 == 0:
                        print("VISSIM 재시작...")
                        Vissim.Exit()  # VISSIM 종료
                        del Vissim
                        time.sleep(5)
                        print("simulation_run_count : ", simulation_run_count)
                        print("senario_cnt : ", senario_cnt)
                        if simulation_run_count >= senario_cnt:
                            print("시뮬레이션이 완료되었습니다.")
                        else:
                            Vissim = win32com.client.Dispatch("Vissim.Vissim")  # VISSIM 다시 실행
                            print("VISSIM 재시작 완료")

file_path :  C:\VISSIM_Workspace\network_new\network_version2\network_version4.1\network02_수정.inpx
new_gradient :  -0.03 new_gradient2 :  0.03
100rel:  0.7262 300rel :  0.0738 630rel :  0.1426 640rel :  0.0507 650rel :  0.0068 volume :  1356
VISSIM 시뮬레이션 실행 중...
VISSIM 시뮬레이션 종료
results_files :  ['1.db', '2.db', '3.db', '4.db', '5.db', '6.db', '7.db', '8.db', '9.db', '10.db']
batch files :  ['1.db', '2.db', '3.db', '4.db', '5.db', '6.db', '7.db', '8.db', '9.db', '10.db']
db_path :  C:\VISSIM_Workspace\network_new\network_version2\network_version4.1\network02_수정.results\1.db
db_path :  C:\VISSIM_Workspace\network_new\network_version2\network_version4.1\network02_수정.results\2.db
db_path :  C:\VISSIM_Workspace\network_new\network_version2\network_version4.1\network02_수정.results\3.db
db_path :  C:\VISSIM_Workspace\network_new\network_version2\network_version4.1\network02_수정.results\4.db
db_path :  C:\VISSIM_Workspace\network_new\network_version2\network_version4.1\network02_수정.results\5.db
